In [ ]:
%load_ext aiida
%aiida
from IPython.display import display, clear_output, Markdown
import ipywidgets as ipw
import widgets
import utils
import aiida_utils
import json
import shutil
import subprocess

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])
SIMULATION_OBJECT_TYPES = [
    "GEOMETRY_OPTIMISATION",
    "2D_MEASUREMENT",
    "BAND_STRUCTURE",
    "PDOS",
    "VIBRATIONAL_SPECTROSCOPY"
]

WORKCHAIN_VIEWERS = {
    'QeAppWorkChain': "/apps/apps/quantum-espresso/qe.ipynb",
    'Cp2kGeoOptWorkChain': "/apps/apps/surfaces/view_geometry_optimization.ipynb",
    'Cp2kStmWorkChain': "/apps/apps/surfaces/view_stm.ipynb",
}

slabs_options = [object_key for object_key, object_info in CONFIG["objects"].items() if object_info["object_type"] == "slab"]
slabs_options.insert(0, "No material")
material_selection_radio_button = utils.Radiobuttons(
    description = '', options = slabs_options, 
    disabled = False, layout = ipw.Layout(width = '300px'), 
    style = {'description_width': "100px"}
)

material_selector = widgets.MaterialSelectionWidget()

experiment_selector = widgets.ExperimentSelectionWidget()
experiment_selector.load_dropdown_box()

molecule_selector_list = widgets.MultipleSelectorWidget("molecule")
molecule_selector_list_output = ipw.Output()

add_molecule_button = utils.Button(
    description = 'Add molecule', disabled = False, button_style = '', 
    tooltip = 'Add molecule', layout = ipw.Layout(width = '150px', height = '25px')
)
remove_molecule_button = utils.Button(
    description = 'Remove molecule', disabled = False, button_style = '', 
    tooltip = 'Remove molecule', layout = ipw.Layout(width = '150px', height = '25px')
)
add_remove_molecule_buttons_hbox = ipw.HBox([add_molecule_button, remove_molecule_button])

product_selector_list = widgets.MultipleSelectorWidget("product")
product_selector_list_output = ipw.Output()

add_product_button = utils.Button(
    description = 'Add product', disabled = False, button_style = '', 
    tooltip = 'Add product', layout = ipw.Layout(width = '150px', height = '25px')
)
remove_product_button = utils.Button(
    description = 'Remove product', disabled = False, button_style = '', 
    tooltip = 'Remove product', layout = ipw.Layout(width = '150px', height = '25px')
)
add_remove_product_buttons_hbox = ipw.HBox([add_product_button, remove_product_button])

logical_operator_dropdown = utils.Dropdown(
    description = "Search logical operator", 
    disabled = False, 
    layout = ipw.Layout(width = "300px"), 
    options = ["AND", "OR"],
    value = "AND",
    style = {'description_width': "140px"}
)

search_results_output = ipw.Output()

found_simulations_selector = utils.SelectMultiple(
    description = 'Simulations', disabled = False, 
    layout = ipw.Layout(width = '800px'), 
    style = {'description_width': "110px"}
)

search_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Search', 
    icon = 'search', layout = ipw.Layout(width = '50px', height = '25px')
)

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
download_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'download', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
download_close_buttons_hbox = ipw.HBox([download_button, quit_button])

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))
    
def select_material_radio_change(change):
    material_type = material_selection_radio_button.value
    if material_type == "No material":
        with material_selector:
            clear_output()
            return
    
    material_selector.details_textbox.value = ''
    material_types = {}
    for object_key, object_info in CONFIG["objects"].items():
        if object_info["object_type"] == "slab":
            material_types[object_key] = (object_info["openbis_object_type"], object_info["placeholder"])
            
    with material_selector:
        clear_output()
        display_list = [
            material_selector.dropdown_boxes,
            ipw.HBox([material_selector.details_textbox, material_selector.image_box])
        ]
        material_class, placeholder = material_types.get(material_type)
        material_selector.load_dropdown_box(material_class, placeholder)
        display(ipw.VBox(display_list))

# Function to handle changes in the materials dropdown
def load_material_metadata(change):
    if material_selector.dropdown_boxes.children[0].value == -1:
        material_selector.details_textbox.value = ''
        material_selector.image_box.value = utils.read_file(CONFIG["default_image_filepath"])
        return
    
    # Get selected object properties information from config file
    selected_object = material_selection_radio_button.value
    selected_object_properties = CONFIG["objects"][selected_object]["properties"]
    
    # Get material object information and dataset
    material_object = OPENBIS_SESSION.get_object(material_selector.dropdown_boxes.children[0].value)
    material_dataset = material_object.get_datasets()[0]
    
    # Get the object image preview
    if material_dataset:
        material_dataset.download(destination="images")
        material_selector.image_box.value = utils.read_file(f"images/{material_dataset.permId}/{material_dataset.file_list[0]}")
        # Erase file after downloading it
        shutil.rmtree(f"images/{material_dataset.permId}")
    else:
        material_selector.image_box.value = utils.read_file(CONFIG["default_image_filepath"])

    # Make a string with the property values of the object
    material_metadata = material_object.props.all()
    material_metadata_string = ""
    for prop_key in selected_object_properties:
        prop_title = CONFIG["properties"][prop_key]["title"]
        if CONFIG["properties"][prop_key]["property_type"] == "QUANTITY_VALUE":
            value = material_metadata.get(prop_key)
            if value:
                prop_dict = json.loads(value)
                material_metadata_string += f"{prop_title}: {prop_dict['value']} {prop_dict['unit']}\n"
            else:
                material_metadata_string += f"{prop_title}: {value}\n"
        else:
            material_metadata_string += f"{prop_title}: {material_metadata.get(prop_key)}\n"

    material_selector.details_textbox.value = material_metadata_string

def add_molecule_widget(b):
    molecule_selector_list.add_selector()
    with molecule_selector_list_output:
        clear_output()
        display(molecule_selector_list)

def remove_molecule_widget(b):
    molecule_selector_list.remove_selector()
    with molecule_selector_list_output:
        clear_output()
        display(molecule_selector_list)

def add_product_widget(b):
    product_selector_list.add_selector()
    with product_selector_list_output:
        clear_output()
        display(product_selector_list)

def remove_product_widget(b):
    product_selector_list.remove_selector()
    with product_selector_list_output:
        clear_output()
        display(product_selector_list)

def search_simulations(b):
    parents_permid_list = []
    if molecule_selector_list.selectors:
        for molecule_selector in molecule_selector_list.selectors:
            if molecule_selector.dropdown.value != -1:
                parents_permid_list.append(molecule_selector.dropdown.value)
    if material_selector.dropdown.value != -1:
        parents_permid_list.append(material_selector.dropdown.value)
    if product_selector_list.selectors:
        for product_selector in product_selector_list.selectors:
            if product_selector.dropdown.value != -1:
                parents_permid_list.append(product_selector.dropdown.value)
    
    simulation_permid_set = set()
    
    if logical_operator_dropdown.value == "OR": # In OR, all the simulations found are added to the list
        for parent in parents_permid_list:
            parent_object = OPENBIS_SESSION.get_sample(parent)
            simulation_objects_children = find_openbis_simulations(parent_object)
            for simulation_object in simulation_objects_children:
                simulation_permid = simulation_object.permId
                if simulation_object.type == "2D_MEASUREMENT": # 2D Measurement is used for both simulation and experiments
                    simulation_measurement = False
                    for parent in simulation_object.parents:
                        parent_object = OPENBIS_SESSION.get_sample(parent)
                        if parent_object.type == "ATOMISTIC_MODEL":
                            simulation_measurement = True
                    if simulation_measurement:
                        simulation_permid_set.add(simulation_permid)
                else:
                    simulation_permid_set.add(simulation_permid)
    else:                                       # In AND, only the simulations that appear in all selected materials are added to the list
        for idx, parent in enumerate(parents_permid_list):
            parent_object = OPENBIS_SESSION.get_sample(parent)
            simulation_objects_children = find_openbis_simulations(parent_object)
            parent_simulation_permid_list = []
            for simulation_object in simulation_objects_children:
                simulation_permid = simulation_object.permId
                if simulation_object.type == "2D_MEASUREMENT": # 2D Measurement is used for both simulation and experiments
                    simulation_measurement = False
                    for parent in simulation_object.parents:
                        parent_object = OPENBIS_SESSION.get_sample(parent)
                        if parent_object.type == "ATOMISTIC_MODEL":
                            simulation_measurement = True
                    if simulation_measurement:
                        parent_simulation_permid_list.append(simulation_permid)
                else:
                    parent_simulation_permid_list.append(simulation_permid)
            
            if idx == 0:   
                simulation_permid_set = set(parent_simulation_permid_list)
            else:
                simulation_permid_set.intersection_update(parent_simulation_permid_list)

    simulation_permid_list = list(simulation_permid_set)
    aiida_node_permid_list = []
    for simulation_permid in simulation_permid_list:
        simulation_object = OPENBIS_SESSION.get_sample(simulation_permid)
        for parent in simulation_object.parents:
            parent_object = OPENBIS_SESSION.get_sample(parent)
            if parent_object.type == "AIIDA_NODE":
                aiida_node_permid_list.append(parent)
    
    simulation_aiida_node_list = []
    for idx, simulation_permid in enumerate(simulation_permid_list):
        simulation_object = OPENBIS_SESSION.get_sample(simulation_permid)
        simulation_info = f"{simulation_object.props['$name']}"
        aiida_node_permid = aiida_node_permid_list[idx]
        simulation_aiida_node_list.append([simulation_info, aiida_node_permid])
    
    with search_results_output:
        clear_output()
        found_simulations_selector.options = simulation_aiida_node_list
        display(found_simulations_selector)

def find_openbis_simulations(object):
    simulation_objects = set()
    children = object.children
    if children is not None:
        for child in children:
            child_object = OPENBIS_SESSION.get_sample(child)
            if child_object.type in SIMULATION_OBJECT_TYPES:
                simulation_objects.add(child_object)
            simulation_objects.update(find_openbis_simulations(child_object))
    return simulation_objects

def import_aiida_nodes(b):
    selected_values = found_simulations_selector.value
    selected_labels = [label for label, value in found_simulations_selector.options if value in selected_values]
    for idx, aiida_node_permid in enumerate(selected_values):
        selected_label = selected_labels[idx]
        aiida_node_object = OPENBIS_SESSION.get_sample(aiida_node_permid)
        object_datasets = aiida_node_object.get_datasets()
        for dataset in object_datasets:
            dataset_filenames = dataset.file_list
            is_aiida_file = False
            if len(dataset_filenames) == 1:
                for filename in dataset_filenames:
                    if ".aiida" in filename:
                        is_aiida_file = True
            
            if is_aiida_file:
                dataset.download(destination = 'aiida_nodes')
                aiida_node_filename = dataset.file_list[0]
                aiida_node_filepath = f"aiida_nodes/{dataset.permId}/{aiida_node_filename}"
                command = ["verdi", "archive", "import", aiida_node_filepath]
                
                # Execute the command
                result = subprocess.run(command, capture_output=True, text=True)
                if result.returncode != 0:
                    print(f"An error occurred: {result.stderr}")
                else:
                    workchain = load_node(aiida_node_object.props["wfms_uuid"])
                    workchain_viewer_link = WORKCHAIN_VIEWERS[workchain.process_label]
                    notebook_link = f"{workchain_viewer_link}?pk={workchain.pk}"
                    display(Markdown(f"[Workchain {selected_label} successfully imported.]({notebook_link})"))
                shutil.rmtree(f"aiida_nodes/{dataset.permId}/")

# Import simulations from openBIS into Aiidalab

## Select molecule

In [ ]:
display(molecule_selector_list_output)
with molecule_selector_list_output:
    display(molecule_selector_list)  
display(add_remove_molecule_buttons_hbox)

## Select reaction product

In [ ]:
display(product_selector_list_output)
with product_selector_list_output:
    display(product_selector_list)  
display(add_remove_product_buttons_hbox)

## Select slab

In [ ]:
display(material_selection_radio_button)
display(material_selector)

## Search results

In [ ]:
display(ipw.HBox([logical_operator_dropdown, search_button]))
display(search_results_output)

In [ ]:
display(download_close_buttons_hbox)
display(increase_buttons_size)
material_selection_radio_button.observe(select_material_radio_change, names='value')
material_selector.dropdown.observe(load_material_metadata, names = 'value')
add_molecule_button.on_click(add_molecule_widget)
remove_molecule_button.on_click(remove_molecule_widget)
add_product_button.on_click(add_product_widget)
remove_product_button.on_click(remove_product_widget)
search_button.on_click(search_simulations)
download_button.on_click(import_aiida_nodes)
quit_button.on_click(close_notebook)